## **1) Importing Python Packages for GAN**


In [1]:
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Dropout
from keras.layers import ELU, PReLU, LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir generated_images

## **2) Parameters for Neural Networks & Data**

In [3]:
img_width = 32
img_height = 32
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0002)

## **3) Building Generator**





In [4]:
def build_generator():
    model = Sequential()

    # Create first layer, to receive the input 
    model.add(Dense(256 * 4 * 4, input_dim = latent_dim))
    # 256 * 8 * 8; for upscaling the layers, 
    # initial shape to construct into final shape

    # Create default activation function
    model.add(LeakyReLU(alpha = 0.2))

    # Create reshape layer
    model.add(Reshape((4, 4,256)))
    # 8,8,256 ; reffers to first layer

    # Adding more layers for neurons and better result
    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha= 0.2))
    # (4,4) >> filter size
    # strides = (2,2) >> Convolutional layers, that how NN understand images

    # Create Final output layer and forming image shape
    # the shape (3, (3,3)) reffers to image shape :
    #    >>>  img_shape = (img_width, img_height, channels)
    model.add(Conv2D(3, (3,3), activation= 'tanh', padding = 'same'))

    #
    model.summary()
    return model

generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4096)              413696    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 4096)              0         
                                                                 
 reshape (Reshape)           (None, 4, 4, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 128)        524416    
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 16, 16, 128)      262272    
 ranspose)                                              

## **4) Building Discriminator**

In [5]:
def build_discriminator():
    model = Sequential()

    # Create input layer and filter and stride layer. That makes NN understand image
    model.add(Conv2D(64, (3,3), padding = 'same', input_shape = img_shape))

    # Adding activation function
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Conv2D(128, (3,3), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Conv2D(128, (3,3), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Conv2D(256, (3,3), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Flatten())

    model.add(Dropout(0.4))

    # Create output layer
    model.add(Dense(1, activation = 'sigmoid'))

    model.summary()
    return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 128)       147584    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 256)      

## **5) Connecting Neural Networks to build GAN**

In [6]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)

In [7]:
GAN.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 3)         1466115   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 780545    
                                                                 
Total params: 2,246,660
Trainable params: 1,466,115
Non-trainable params: 780,545
_________________________________________________________________


## **6) Outputting Images**


In [8]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    # print("%.8f" % save_name)

    # Rescale images 0 - 1
    # gen_imgs = 0.5 * gen_imgs + 0.5
    gen_imgs = (gen_imgs + 1) / 2.0
    # gen_imgs = gen_imgs * 255

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    plt.close()

## **7) Training GAN**

In [ ]:
def train(epochs, batch_size = 64, save_interval = 200):
  (X_train, _), (_, _) = cifar10.load_data()

  # Rescaling the data
  X_train = X_train / 127.5 -1.

  bat_per_epo = int(X_train.shape[0] / batch_size)

  # Create Y label for NN
  valid = np.ones((batch_size,1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range (epochs) :
    for j in range(bat_per_epo) :
      #Get Random Batch
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]

      # Generate Fakes Images
      noise = np.random.normal(0, 1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      # Train Discriminator
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      noise = np.random.normal(0, 1, (batch_size, latent_dim))

      # Inverse Y label
      g_loss = GAN.train_on_batch(noise, valid)

      #print('******* %d %d [D loss: %f, acc: %.2f%%] [G loss: %f]' % (epoch, d_loss[0],100*d_loss[1], g_loss))
      print("******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100* d_loss[1], g_loss))
    save_imgs(epoch)

train(30000, batch_size = 64, save_interval = 200)

2/2 [==============================] - 1s 126ms/step
******* 0 [D loss: 0.701673, acc: 17.19%] [G loss: 0.684214]
2/2 [==============================] - 0s 132ms/step
******* 0 [D loss: 0.497402, acc: 50.00%] [G loss: 0.665537]
2/2 [==============================] - 0s 128ms/step
******* 0 [D loss: 0.423296, acc: 50.00%] [G loss: 0.630734]
2/2 [==============================] - 0s 139ms/step
******* 0 [D loss: 0.421896, acc: 50.00%] [G loss: 0.598685]
2/2 [==============================] - 0s 106ms/step
******* 0 [D loss: 0.440285, acc: 50.00%] [G loss: 0.596709]
2/2 [==============================] - 0s 146ms/step
******* 0 [D loss: 0.424134, acc: 50.00%] [G loss: 0.656994]
2/2 [==============================] - 0s 98ms/step
******* 0 [D loss: 0.357270, acc: 54.69%] [G loss: 0.818239]
2/2 [==============================] - 0s 126ms/step
******* 0 [D loss: 0.249535, acc: 100.00%] [G loss: 1.162734]
2/2 [==============================] - 0s 150ms/step
******* 0 [D loss: 0.149344, acc: 1

In [ ]:
noise = np.random.normal(0, 1, (1,latent_dim))
gen_imgs = generator.predict(noise)

In [ ]:
gen_imgs = (gen_imgs + 1) / 2.0
plt.imshow(gen_imgs)

### **8) Making GIF**

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)